# Image matching with Siamese networks
## Dependencies

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import torch
from torch import optim
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch.utils.data import DataLoader
import utilities as utils 
from siameseDataloader import readDataFolder, dataSplits, TripletDataset
from siameseModel import MobileNetTriplet

In [ ]:
import os
config = utils.load_config()
datadir = os.path.join('data','tiny-imagenet-200-copy','train')#config['DATASET']['root']
numclasses = config['DATASET']['numclasses']
sameprob = config['DATASET']['sameprob']
im_size = config['DATASET']['im_size']
emb_size=config['MODEL']['embsize']

seed = config['seed']
np.random.seed(seed)

device = config['TRAIN']['device']

## Dataset

In [ ]:
dataset = readDataFolder(datadir,numclasses)
print(dataset)

In [ ]:
train_split,val_split,test_split = dataSplits(dataset,0.8,0.2,0)

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize(im_size+4),
    transforms.RandomCrop(im_size),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(0.3),
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    transforms.Normalize([0.507, 0.4865, 0.4409], [0.2673, 0.2564, 0.2761])
])
test_transforms = transforms.Compose([
    transforms.Resize(im_size),
    transforms.CenterCrop(im_size),
    transforms.ToTensor(),
    #transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])
    transforms.Normalize([0.507, 0.4865, 0.4409], [0.2673, 0.2564, 0.2761])
])

train_dataset = TripletDataset(train_split,train_transforms)
val_dataset = TripletDataset(val_split,test_transforms)
test_dataset = TripletDataset(test_split,test_transforms)

datasets = {'train':train_dataset,'val':val_dataset,'test':test_dataset}

In [ ]:
#dataloaders
dataloaders = {
    'train': DataLoader(train_dataset,shuffle=True,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize']),
    'val': DataLoader(val_dataset,shuffle=False,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize']),
    'test': DataLoader(test_dataset,shuffle=False,
                        num_workers=config['TRAIN']['numworkers'],
                        batch_size=config['TRAIN']['batchsize'])
}

In [ ]:
for i,data in enumerate(dataloaders['train']):
    print(data[0].shape)
    break

## Model

In [ ]:
model = MobileNetTriplet(config['MODEL']['distance'],embsize=emb_size,pretrained=config['MODEL']['pretrained'])

In [ ]:
for param in model.parameters():
    param.requires_grad = True

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

In [ ]:
data = train_dataset[400]
out = model(data[0].reshape(1,3,im_size,im_size),data[1].reshape(1,3,im_size,im_size),data[2].reshape(1,3,im_size,im_size))
print(out)
#print(criterion(out,data[2]))


## Train

In [ ]:
#model.to('cpu')
#model.load_state_dict(model1)

In [ ]:
criterion = torch.nn.TripletMarginWithDistanceLoss(margin = config['TRAIN']['lossmargin'], distance_function=model.distance)
#config['TRAIN']['lr'] =0.0005
optimizer = optim.Adam(model.parameters(),lr = config['TRAIN']['lr'],weight_decay=0.03)
#scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:

model = model.to(device)

In [ ]:
loss_history ={'train':[],'val':[]}
minloss=10000


In [ ]:
best_model = model.state_dict().copy() 

In [ ]:

for epoch in range(config['TRAIN']['numepochs']):
    for mode in ['train','val']:
        loss_epoch=0
        count=0
        for i,(img1,img2,img3) in enumerate(dataloaders[mode]):
            img1 = img1.to(device)
            img2 = img2.to(device)
            img3 = img3.to(device)
            
            if mode=='train':
                model.train()
                
                optimizer.zero_grad()
                out1,out2,out3 = model(img1,img2,img3)
                loss = criterion(out1,out2,out3)
                loss.backward()
                optimizer.step()
                
            else:
                model.eval()
                with torch.no_grad():
                    out1,out2,out3 = model(img1,img2,img3)
                loss = criterion(out1,out2,out3)
                print('val loss')
                print(loss)
            
            # track total loss
            loss_epoch = loss_epoch+loss
            count = count + img1.shape[0]
            
        #scheduler.step()
        
        loss_history[mode].append(loss_epoch.item()/count)
    print('Epoch: {},\t train loss:{:.5},\t val loss:{:.5}'.format(epoch,loss_history['train'][-1],loss_history['val'][-1]))

    if (loss_history['val'][-1] < minloss):
        minloss = loss_history['val'][-1]
        best_model = model.state_dict().copy()
            
            

In [ ]:
model.load_state_dict(best_model)

In [ ]:
plt.figure()
plt.plot(loss_history['train'],'b-')
plt.plot(loss_history['val'],'r-')

## Train acc

In [ ]:
model.eval()
model=model.to(device)
#threshs = np.linspace(start=0,stop=config['TRAIN']['lossmargin']*1.2,num=10)
threshs = [config['TRAIN']['lossmargin']/2]
#threshs=[0.5]
accs = []
sensitivity = []
specificity = []
f1=[]


for thresh in threshs:
    pos=0
    neg=0
    conf = np.zeros((2,2))
    for i,(img1,img2,label) in enumerate(dataloaders['train']):
        img1=img1.to(device)
        img2=img2.to(device)
        label=label.to(device)
        with torch.no_grad():
            out = model(img1,img2)
        
        conf[1,1] += torch.matmul(label.T*1.,(out>=thresh)*1.)
        conf[1,0] += torch.matmul(label.T*1.,(out<thresh)*1.)
        conf[0,1] += torch.matmul((1-label).T*1.,(out>=thresh)*1.)
        conf[0,0] += torch.matmul((1-label).T*1.,(out<thresh)*1.)
        pos = pos + torch.sum(label)
        neg = neg + torch.sum(1-label)

    accs.append(((conf[1,1]+conf[0,0])/(pos+neg)).item())
    sensitivity.append((conf[0,0]/neg).item()) #changed 0 and 1 labels (0 is true in the loss fcn)
    specificity.append((conf[1,1]/pos).item())
    f1.append(((2*conf[0,0])/(2*conf[0,0]+conf[0,1]+conf[1,0])).item())

In [ ]:
print(accs)
print(f1)

## Val

In [ ]:
loss_epoch = 0
count=0
for i,(img1,img2,label) in enumerate(dataloaders['val']):
    img1 = img1.to(device)
    img2=img2.to(device)
    label = label.to(device)
    with torch.no_grad():
        out = model(img1,img2)
        loss = criterion(out,label)

    # track total loss
    loss_epoch = loss_epoch+loss
    count = count + len(label)
print('Validation loss: {}'.format(loss_epoch.item()/count))

In [ ]:
# Visualise
numex=5 #number of examples
fig,ax = plt.subplots(2,numex,figsize=(15,5))
for i in range(numex):
    example = val_dataset[i]
    utils.imshow(example[0],ax[0][i])
    utils.imshow(example[1],ax[1][i])
    distance = model(example[0].reshape(1,3,im_size,im_size).to(device),example[1].reshape(1,3,im_size,im_size).to(device))
    ax[0][i].set_title('label:{},distance:{:.2}'.format(example[2].item(),distance.item()))
plt.show()

In [ ]:
#threshs = np.linspace(start=0,stop=config['TRAIN']['lossmargin']*1.2,num=11)
threshs = [config['TRAIN']['lossmargin']/2]
#threshs = [0.5]
#threshs = np.linspace(start=0,stop=1,num=11)
accs=[]
sensitivity = []
specificity = []
f1=[]

num=0
for thresh in threshs:
    pos=0
    neg=0
    conf = np.zeros((2,2))
    for i,(img1,img2,label) in enumerate(dataloaders['val']):
        img1=img1.to(device)
        img2=img2.to(device)
        label=label.to(device)
        
        with torch.no_grad():
            out = model(img1,img2)
        
        conf[1,1] += torch.matmul(label.T*1.,(out>=thresh)*1.)
        conf[1,0] += torch.matmul(label.T*1.,(out<thresh)*1.)
        conf[0,1] += torch.matmul((1-label).T*1.,(out>=thresh)*1.)
        conf[0,0] += torch.matmul((1-label).T*1.,(out<thresh)*1.)
        pos = pos + torch.sum(label)
        neg = neg + torch.sum(1-label)

    accs.append(((conf[1,1]+conf[0,0])/(pos+neg)).item())
    sensitivity.append((conf[0,0]/neg).item()) #changed 0 and 1 labels (0 is true in the loss fcn)
    specificity.append((conf[1,1]/pos).item())
    f1.append(((2*conf[0,0])/(2*conf[0,0]+conf[0,1]+conf[1,0])).item())
    
    print(num)
    num += 1

In [ ]:
print(accs)
print(f1)

In [ ]:
accs = np.array(accs)
sensitivity = np.array(sensitivity)
specificity = np.array(specificity)
f1 = np.array(f1)

m1 = np.argmax(accs)
m2 = np.argmax(f1)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4))
ax[0].plot(threshs,accs)
ax[0].set_xlabel('threshold')
ax[0].set_ylabel('accuracy')
ax[1].plot(threshs,f1)
ax[1].set_xlabel('threshold')
ax[1].set_ylabel('f1-score')
ax[2].plot(1-specificity,sensitivity,'o',label='_nolabel_')
ax[2].set_xlabel('1-specificity')
ax[2].set_ylabel('sensitivity')
ax[2].plot(1-specificity[m1],sensitivity[m1],'ro',label='acc thresh:{}'.format(threshs[m1]))
ax[2].plot(1-specificity[m2],sensitivity[m2],'go',label='f1 thresh:{}'.format(threshs[m2]))
ax[2].legend()

## Test

In [ ]:
loss_epoch=0
count=0
for i,(img1,img2,label) in enumerate(dataloaders['test']):
    with torch.no_grad():
        out = model(img1,img2)
        loss = criterion(out,label)

    # track total loss
    loss_epoch = loss_epoch+loss
    count = count + len(label)
print('Test loss: {}'.format(loss_epoch.item()/count))
    

In [ ]:
# Visualise
numex=5 #number of examples
fig,ax = plt.subplots(2,numex,figsize=(15,5))
for i in range(numex):
    example = test_dataset[i]
    utils.imshow(example[0],ax[0][i])
    utils.imshow(example[1],ax[1][i])
    distance = model(example[0].reshape(1,3,64,64),example[1].reshape(1,3,64,64))
    ax[0][i].set_title('label:{},distance:{:.2}'.format(example[2].item(),distance.item()))
plt.show()

In [ ]:
threshs = [1]
accs = []
sensitivity = []
specificity = []
f1=[]


for thresh in threshs:
    pos=0
    neg=0
    conf = np.zeros((2,2))
    for i,(img1,img2,label) in enumerate(dataloaders['test']):
        with torch.no_grad():
            out = model(img1,img2)
        
        conf[1,1] += torch.matmul(label.T,(out>=thresh)*1)
        conf[1,0] += torch.matmul(label.T,(out<thresh)*1)
        conf[0,1] += torch.matmul((1-label).T,(out>=thresh)*1)
        conf[0,0] += torch.matmul((1-label).T,(out<thresh)*1)
        pos = pos + torch.sum(label)
        neg = neg + torch.sum(1-label)

    accs.append(((conf[1,1]+conf[0,0])/(pos+neg)).item())
    sensitivity.append((conf[0,0]/neg).item()) #changed 0 and 1 labels (0 is true in the loss fcn)
    specificity.append((conf[1,1]/pos).item())
    f1.append(((2*conf[0,0])/(2*conf[0,0]+conf[0,1]+conf[1,0])).item())
    break

In [ ]:
accs = np.array(accs)
sensitivity = np.array(sensitivity)
specificity = np.array(specificity)
f1 = np.array(f1)

m1 = np.argmax(accs)
m2 = np.argmax(f1)

In [ ]:
fig,ax = plt.subplots(1,3,figsize=(15,4))
ax[0].plot(threshs,accs)
ax[0].set_xlabel('threshold')
ax[0].set_ylabel('accuracy')
ax[1].plot(threshs,f1)
ax[1].set_xlabel('threshold')
ax[1].set_ylabel('f1-score')
ax[2].plot(1-specificity,sensitivity,'o',label='_nolabel_')
ax[2].set_xlabel('1-specificity')
ax[2].set_ylabel('sensitivity')
ax[2].plot(1-specificity[m1],sensitivity[m1],'ro',label='acc thresh:{}'.format(threshs[m1]))
ax[2].plot(1-specificity[m2],sensitivity[m2],'go',label='f1 thresh:{}'.format(threshs[m2]))
ax[2].legend()

## Save model

In [ ]:
#torch.save(model.state_dict(),config['MODEL']['modelpath'])
model = model.to('cpu')
torch.save(model.state_dict(),"models\\mobilenet_state_dict_50cls.pkl")

# Exctract embeddings

In [ ]:
model = model.to(device)

In [ ]:
train_embs = []
train_labels = []
val_embs = []
val_labels = []

In [ ]:
for i,(img1,img2,label) in enumerate(dataloaders['train']):
    img1 = img1.to(device)
    img2 = img2.to(device)
    with torch.no_grad():
        out = model.getEmb(img1,img2)

    train_embs+=out.tolist()
    train_labels+=label.detach().tolist()

In [ ]:
for i,(img1,img2,label) in enumerate(dataloaders['val']):
    img1 = img1.to(device)
    img2 = img2.to(device)
    with torch.no_grad():
        out = model.getEmb(img1,img2)

    val_embs+=out.tolist()
    val_labels+=label.detach().tolist()

In [ ]:
torch.save((train_embs,train_labels),'trainembs.pkl')
torch.save((val_embs,val_labels),'valembs.pkl')